In [1]:
from data import load_data, SOURCES, save_to_json, load_from_json
from models import (
    get_gpt_recognition_logprobs,
    get_model_choice,
    get_logprobs_choice_with_sources,
    get_gpt_score,
    get_gpt_summary_similarity,
    get_gpt_paraphrase,
    get_gpt_choice_null_baseline
)

from math import exp
from pprint import pprint
from random import shuffle

from tqdm import tqdm
import time
import os

import random
import nltk
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize

import re
import json

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

from sentence_transformers import SentenceTransformer, util
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np



c:\Users\G25971483\Desktop\Projects\LLM\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
response

ChatCompletion(id='chatcmpl-ATjvcd6TmwP3SpMZgz5ELc2TdzE9N', choices=[Choice(finish_reason='stop', index=0, logprobs=ChoiceLogprobs(content=[ChatCompletionTokenLogprob(token='3', bytes=[51], logprob=-0.0016588744, top_logprobs=[TopLogprob(token='3', bytes=[51], logprob=-0.0016588744), TopLogprob(token='1', bytes=[49], logprob=-6.4954944), TopLogprob(token='2', bytes=[50], logprob=-8.830752)])], refusal=None), message=ChatCompletionMessage(content='3', refusal=None, role='assistant', function_call=None, tool_calls=None))], created=1731651424, model='gpt-4-1106-preview', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=1, prompt_tokens=212, total_tokens=213, completion_tokens_details=CompletionTokensDetails(reasoning_tokens=0, audio_tokens=0, accepted_prediction_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details={'cached_tokens': 0, 'audio_tokens': 0}))

In [9]:
response.choices[0].message.content

'3'

In [10]:
response.choices[0].logprobs.content[0].top_logprobs

[TopLogprob(token='3', bytes=[51], logprob=-0.0016588744),
 TopLogprob(token='1', bytes=[49], logprob=-6.4954944),
 TopLogprob(token='2', bytes=[50], logprob=-8.830752)]

In [11]:
response.choices[0].logprobs.content[0].top_logprobs[0].token

'3'

In [13]:
# Extract the logprob for the token '3'
logprob_for_3 = next((item.logprob for item in response.choices[0].logprobs.content[0].top_logprobs if item.token == '3'), None)

# Output the result
print(f"The logprob for token '3' is: {logprob_for_3}")
print(f"The prob for token '3' is: {exp(logprob_for_3)}")

The logprob for token '3' is: -0.0016588744
The prob for token '3' is: 0.99834250077162


In [ ]:
def generate_gpt_detect_null(
    dataset,
    model,
    comparison_models,
    starting_idx=0,
    ending_idx = 1000
):
    exact_model = model

    responses, articles, keys = load_data(dataset)

    results = []  

    for key in tqdm(keys[starting_idx:ending_idx], desc="Processing keys"):
        article = articles[key]
        first_summary = responses[comparison_models[0]][key]
        second_summary = responses[comparison_models[1]][key]
        # Shuffle at every key
        summaries = [first_summary, second_summary]
        random.shuffle(summaries)
        first_summary, second_summary = summaries
        
        result = {"key": key}

        try:
            forward_result = get_gpt_choice_null_baseline(
                first_summary,
                second_summary,
                article,
                return_logprobs=True,
            )

            # forward_choice = forward_result[0].token

            logprob_for_3 = next((item.logprob for item in forward_result if item.token == '3'), None)
            result["detection_score"] = exp(logprob_for_3)

        except ValueError:
            print('Error:', key)
        
        results.append(result)
    return results



In [9]:
for model in ["gpt4"]:
    
    print(f"Starting {model}")
    comparison_models =  ["human", "gpt35"]
    print(comparison_models)
    results = generate_gpt_detect_null(
        "cnn", model,comparison_models, starting_idx=500
    )
    #Save results
    file_name = f"{model}_results_null_pair_{comparison_models[0]}_{comparison_models[1]}.json"
    print(file_name)
    path = os.path.join("results","cnn","null_pair",file_name)
    save_to_json(results,path)

Starting gpt4
['human', 'gpt35']


Processing keys: 100%|██████████| 500/500 [06:28<00:00,  1.29it/s] 

gpt4_results_null_pair_human_gpt35.json
